In [1]:
import pandas as pd

In [2]:
ds=pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")

In [3]:
ds

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [4]:
x_train=ds["text"]
y_train=ds["target"]

In [5]:
import re
x_train=x_train.apply(lambda x: re.sub(r'http\S+|www\S+', '', x))

# Remove Hashtags
x_train =x_train.apply(lambda x: re.sub(r'#\w+', '', x))

# Remove Mentions
x_train =x_train.apply(lambda x: re.sub(r'@\w+', '', x))

# Remove extra spaces after cleaning
x_train =x_train.apply(lambda x: ' '.join(x.split()))

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer()
X_train_cv = v.fit_transform(x_train.astype('U').values)
X_train_cv

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 92935 stored elements and shape (7613, 13604)>

In [7]:
X_train_cv.shape

(7613, 13604)

In [8]:
test=pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")
X_test=test["text"]
X_test_cv = v.transform(X_test)


In [9]:

from tensorflow import keras 
import tensorflow as tf
tf.config.optimizer.set_jit(False)  # Disable XLA JIT compilation

nn_model=keras.Sequential([
    keras.layers.Dense(16,input_shape=(13604,),activation="relu"),
    keras.layers.Dense(32,activation="relu"),
    keras.layers.Dense(16,activation="relu"),
    keras.layers.Dense(32,activation="relu"),
    keras.layers.Dense(1,activation="sigmoid")
])


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
nn_model.compile(
optimizer="adam",
loss="binary_crossentropy",
metrics=["accuracy"])

In [11]:
nn_model.fit(X_train_cv, y_train,epochs=10,batch_size=128)

Epoch 1/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6397 - loss: 0.6732
Epoch 2/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8615 - loss: 0.3895
Epoch 3/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9272 - loss: 0.2183
Epoch 4/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9544 - loss: 0.1479
Epoch 5/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9707 - loss: 0.0966
Epoch 6/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9750 - loss: 0.0713
Epoch 7/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9783 - loss: 0.0611
Epoch 8/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9787 - loss: 0.0507
Epoch 9/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9799 - loss: 0.0510
Epoch 10/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9826 - loss: 0.0445


In [12]:
pred=nn_model.predict(X_test_cv)


102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [13]:
outputs=(pred >= 0.5).astype(int).reshape(-1)


In [14]:
outputs


array([1, 1, 1, ..., 1, 1, 1])

In [15]:
sample_submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")
sample_submission["target"] =outputs

In [16]:
sample_submission.describe()

,id,target
count,3263.000000,3263.000000
mean,5427.152927,0.337113
std,3146.427221,0.472796
min,0.000000,0.000000
25%,2683.000000,0.000000
50%,5500.000000,0.000000
75%,8176.000000,1.000000
max,10875.000000,1.000000


In [17]:
sample_submission.to_csv("submission.csv", index=False)